In [ ]:
import time
import json
from selenium import webdriver
#from selenium.webdriver.support.ui import WebDriverWait
#from selenium.webdriver.support import expected_conditions as EC
#from selenium.common.exceptions import ElementClickInterceptedException
#from selenium.webdriver.common.action_chains import ActionChains
#from selenium.webdriver.common.by import By
#from msedge.selenium_tools import Edge, EdgeOptions
from cbc_paths import file_path_html, edge_executable_path, chrome_executable_path, firefox_executable_path

# consider https://github.com/browserup/browserup-proxy
from browsermobproxy import Server
import base64
import brotli

In [ ]:
## https://medium.com/@jiurdqe/how-to-get-json-response-body-with-selenium-amd-browsermob-proxy-71f10335c66

path = 'C:\\Users\\bb\\Downloads\\browsermob\\bin\\browsermob-proxy' #your path to browsermob-proxy
server = Server(path)
server.start()
proxy = server.create_proxy()
profile  = webdriver.FirefoxProfile()
profile.set_proxy(proxy.selenium_proxy())
driver = webdriver.Firefox(executable_path=firefox_executable_path, firefox_profile=profile)
proxy.new_har("file_name", options={'captureHeaders': False, 'captureContent': True, 'captureBinaryContent': True})

cbc_url = 'https://www.cbc.ca/news/politics/trudeau-unvaccinated-canadians-covid-hospitals-1.6305159'
print(cbc_url)
driver.get(cbc_url)

#proxy.wait_for_traffic_to_stop(1, 60)
#print('traffic stopped')

# click the notice button popup to close it
time.sleep(5)
print('1 - sleep 5 seconds complete')
try:
    driver.find_element(By.CLASS_NAME, "noticeButton").click()
    print('clicked notice button')
except:
    print('notice button not found')
    pass

# click show more button first time if available
time.sleep(5)
print('2 - sleep 5 seconds complete')
try:
    print('looking for show_more_button')
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    driver.find_element(By.CLASS_NAME, "vf-load-more").click()
    print('show_more_button clicked first time')
    driver.minimize_window()
except:
    print('show_more_button not found first time')
    pass

counter=1
for ent in proxy.har['log']['entries']:
    counter+=1
    if 'text' in ent['response']['content']:
        response = ent['response']['content']['text']
        # entry["request"]["url"] # see if request url has "json" in it
        try:
            decoded_response = brotli.decompress(base64.b64decode(response)).decode()
            # save to file
            with open('C:\\Users\\bb\\Downloads\\test_json\\data_' + str(counter) + '.txt', 'w', encoding='utf-8') as outfile:
                outfile.write(decoded_response)
            print('COUNTER', str(counter), 'brotli b64decode completed')
        except:
            decoded_response = json.dumps(response, indent=4)
            print('COUNTER', str(counter), 'json.dumps completed')

driver.quit()


In [ ]:
path = 'C:\\Users\\bb\\Downloads\\browsermob\\bin\\browsermob-proxy' #your path to browsermob-proxy
server = Server(path)
server.stop()
driver.quit()


for ent in proxy.har['log']['entries']:
    counter+=1
    if 'text' in ent['response']['content']:
        response = ent['response']['content']['text']
        try:
            decoded_response = brotli.decompress(base64.b64decode(response)).decode()
            # save to file
            with open('C:\\Users\\bb\\Downloads\\test_json\\data_' + str(counter) + '.txt', 'w', encoding='utf-8') as outfile:
                outfile.write(decoded_response)
            print('COUNTER', str(counter), 'brotli b64decode completed')
        except:
            decoded_response = json.dumps(response, indent=4)
            print('COUNTER', str(counter), 'json.dumps completed')